<a href="https://colab.research.google.com/github/Granero0011/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/DS_Unit_1_Sprint_Challenge_2_Data_Wrangling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 2

## Data Wrangling

In this Sprint Challenge you will use data from [Gapminder](https://www.gapminder.org/about-gapminder/), a Swedish non-profit co-founded by Hans Rosling. "Gapminder produces free teaching resources making the world understandable based on reliable statistics."
- [Cell phones (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv)
- [Population (total), by country and year](https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv)
- [Geo country codes](https://github.com/open-numbers/ddf--gapminder--systema_globalis/blob/master/ddf--entities--geo--country.csv)

These two links have everything you need to successfully complete the Sprint Challenge!
- [Pandas documentation: Working with Text Data](https://pandas.pydata.org/pandas-docs/stable/text.html]) (one question)
- [Pandas Cheat Sheet](https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf) (everything else)

## Part 0. Load data

You don't need to add or change anything here. Just run this cell and it loads the data for you, into three dataframes.

In [0]:
import pandas as pd

cell_phones = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--cell_phones_total--by--geo--time.csv')

population = pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--datapoints--population_total--by--geo--time.csv')

geo_country_codes = (pd.read_csv('https://raw.githubusercontent.com/open-numbers/ddf--gapminder--systema_globalis/master/ddf--entities--geo--country.csv')
                       .rename(columns={'country': 'geo', 'name': 'country'}))

## Part 1. Join data

First, join the `cell_phones` and `population` dataframes (with an inner join on `geo` and `time`).

The resulting dataframe's shape should be: (8590, 4)

In [2]:
cell_phones.head()

,geo,time,cell_phones_total
0,abw,1960,0.0
1,abw,1965,0.0
2,abw,1970,0.0
3,abw,1975,0.0
4,abw,1976,0.0


In [3]:
population.head()

,geo,time,population_total
0,afg,1800,3280000
1,afg,1801,3280000
2,afg,1802,3280000
3,afg,1803,3280000
4,afg,1804,3280000


In [5]:
jointtable = pd.merge(cell_phones[["cell_phones_total",'geo', 'time']], 
              population[['geo','time', 'population_total']])
jointtable.shape

(8590, 4)

Then, select the `geo` and `country` columns from the `geo_country_codes` dataframe, and join with your population and cell phone data.

The resulting dataframe's shape should be: (8590, 5)

In [6]:
jointtable.head()

,cell_phones_total,geo,time,population_total
0,0.0,afg,1960,8996351
1,0.0,afg,1965,9938414
2,0.0,afg,1970,11126123
3,0.0,afg,1975,12590286
4,0.0,afg,1976,12840299


In [0]:
jointtable2 = pd.merge(jointtable[["cell_phones_total",'population_total','time','geo']], 
              geo_country_codes[['geo','country']])

In [8]:
jointtable2.shape

(8590, 5)

In [9]:
print(jointtable2)

      cell_phones_total  population_total  time  geo      country
0                   0.0           8996351  1960  afg  Afghanistan
1                   0.0           9938414  1965  afg  Afghanistan
2                   0.0          11126123  1970  afg  Afghanistan
3                   0.0          12590286  1975  afg  Afghanistan
4                   0.0          12840299  1976  afg  Afghanistan
5                   0.0          13067538  1977  afg  Afghanistan
6                   0.0          13237734  1978  afg  Afghanistan
7                   0.0          13306695  1979  afg  Afghanistan
8                   0.0          13248370  1980  afg  Afghanistan
9                   0.0          13053954  1981  afg  Afghanistan
10                  0.0          12749645  1982  afg  Afghanistan
11                  0.0          12389269  1983  afg  Afghanistan
12                  0.0          12047115  1984  afg  Afghanistan
13                  0.0          11783050  1985  afg  Afghanistan
14        

## Part 2. Make features

Calculate the number of cell phones per person, and add this column onto your dataframe.

(You've calculated correctly if you get 1.220 cell phones per person in the United States in 2017.)

In [0]:
import numpy as np
jointtable2['cell_phones_per_person']=pd.Series(np.random.randn(len(jointtable2["cell_phones_total"])), index=jointtable2.index)
for x in range (0, len(jointtable2['cell_phones_per_person'])):
  jointtable2['cell_phones_per_person'][x]=jointtable2["cell_phones_total"][x]/jointtable2['population_total'][x]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


Modify the `geo` column to make the geo codes uppercase instead of lowercase.

In [11]:
jointtable2['geo']=jointtable2['geo'].str.title()
print(jointtable2.head())

   cell_phones_total  population_total  time  geo      country
0                0.0           8996351  1960  Afg  Afghanistan
1                0.0           9938414  1965  Afg  Afghanistan
2                0.0          11126123  1970  Afg  Afghanistan
3                0.0          12590286  1975  Afg  Afghanistan
4                0.0          12840299  1976  Afg  Afghanistan


## Part 3. Process data

Use the describe function, to describe your dataframe's numeric columns, and then its non-numeric columns.

(You'll see the time period ranges from 1960 to 2017, and there are 195 unique countries represented.)

In [13]:
jointtable2['cell_phones_total'].describe()

count    8.590000e+03
mean     9.004950e+06
std      5.573408e+07
min      0.000000e+00
25%      0.000000e+00
50%      6.200000e+03
75%      1.697652e+06
max      1.474097e+09
Name: cell_phones_total, dtype: float64

In [19]:
jointtable2['cell_phones_total'].value_counts()

0.0           3755
5000.0           9
300.0            9
30000.0          9
1500.0           8
2500.0           7
25000.0          7
10000.0          7
1000.0           7
15000.0          7
500.0            6
3500.0           6
4000.0           6
200000.0         6
100000.0         5
150000.0         5
12000.0          5
2300.0           5
1200.0           5
20000.0          5
75000.0          5
700.0            4
2200.0           4
65000.0          4
2000.0           4
1300.0           4
6500.0           4
2000000.0        4
600000.0         4
80000.0          4
              ... 
2020637.0        1
1698536.0        1
1370.0           1
8618.0           1
23940973.0       1
211614.0         1
93666088.0       1
87000.0          1
977521.0         1
993902.0         1
12494391.0       1
23270.0          1
8562000.0        1
13633167.0       1
135000.0         1
6378095.0        1
8068.0           1
24402.0          1
236175.0         1
28190.0          1
1026611.0        1
447769.0    

In [14]:
jointtable2['population_total'].describe()

count    8.590000e+03
mean     2.983823e+07
std      1.161284e+08
min      4.433000e+03
25%      1.456148e+06
50%      5.725062e+06
75%      1.810581e+07
max      1.409517e+09
Name: population_total, dtype: float64

In [18]:
jointtable2['population_total'].value_counts().describe()

count    8589.000000
mean        1.000116
std         0.010790
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         2.000000
Name: population_total, dtype: float64

In [20]:
jointtable2['time'].describe()

count    8590.000000
mean     1994.193481
std        14.257975
min      1960.000000
25%      1983.000000
50%      1995.000000
75%      2006.000000
max      2017.000000
Name: time, dtype: float64

In [24]:
jointtable2['time'].value_counts()
print(len(jointtable2['time'].value_counts()))

46


In [25]:
jointtable2['geo'].describe()

count     8590
unique     195
top        Lux
freq        46
Name: geo, dtype: object

In [26]:
jointtable2['country'].describe()

count        8590
unique        195
top       Romania
freq           46
Name: country, dtype: object

In 2017, what were the top 5 countries with the most cell phones total?

Your list of countries should have these totals:

| country | cell phones total |
|:-------:|:-----------------:|
|    ?    |     1,474,097,000 |
|    ?    |     1,168,902,277 |
|    ?    |       458,923,202 |
|    ?    |       395,881,000 |
|    ?    |       236,488,548 |



In [0]:
# This optional code formats float numbers with comma separators
pd.options.display.float_format = '{:,}'.format

In [40]:
df = pd.DataFrame([jointtable2['country'], jointtable2['cell_phones_total']],columns='cell_phones_total','country')
df.head()

SyntaxError: ignored

In [38]:
df

AttributeError: ignored

2017 was the first year that China had more cell phones than people.

What was the first year that the USA had more cell phones than people?

## Part 4. Reshape data

Create a pivot table:
- Columns: Years 2007—2017
- Rows: China, India, United States, Indonesia, Brazil (order doesn't matter)
- Values: Cell Phones Total

The table's shape should be: (5, 11)

#### OPTIONAL BONUS QUESTION!

Sort these 5 countries, by biggest increase in cell phones from 2007 to 2017.

Which country had 935,282,277 more cell phones in 2017 versus 2007?

If you have the time and curiosity, what other questions can you ask and answer with this data?